In [97]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

import os

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from scipy.stats import boxcox
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC

from sklearn.metrics import log_loss, accuracy_score, f1_score, plot_confusion_matrix, confusion_matrix

from imblearn.over_sampling import SMOTE

seed = 1

le = LabelEncoder()

In [98]:
path = 'C:\\Users\\sunil\\Projects\\Machine Hack\\Merchandise Popularity Prediction\\Dataset'

train = pd.read_csv(path + '\\Train.csv')
test = pd.read_csv(path + '\\Test.csv')
sample_sub = pd.read_csv(path + '\\sample_submission.csv')

In [99]:
target = 'popularity'
features = [col for col in train.columns if col not in [target]]

train[target].replace({0:0, 1:1, 3:2, 4:3, 5:4}, inplace = True)

In [101]:
#removing lowest in Score_2
train.drop(15372, axis = 0, inplace = True)

In [102]:
test.loc[ test['Score_2'] == 0, 'Score_2'] = 0.1

In [103]:
df = pd.concat( [train, test], axis = 0).reset_index(drop=True)

In [104]:
df['Score_2'] = np.log(df['Score_2'])
df['time'] = np.log(df['time'])

In [106]:
df.loc[df['Store_Score']<0, 'Store_Score'] = df[df['Store_Score']<0]['Store_Score']*-1
df['Store_Score'] = np.log(df['Store_Score'])

In [108]:
train, test = df[:train.shape[0]].copy(), df[train.shape[0]:].copy()
test = test.reset_index(drop=True)

In [109]:
def join(train, test):
    df = pd.concat([train, test], axis = 0).reset_index(drop=True)
    return df

def split(df):
    train_new, test_new = df[:train.shape[0]], df[train.shape[0]:]
    feats = [col for col in train_new.columns if col not in [target]]
    return train_new, test_new, feats

In [110]:
df = join(train,test)

In [111]:
df['mean_presence_per_store_score'] = df.groupby('Store_Score')['Store_Presence'].transform('mean')
df['max_presence_per_store'] = df.groupby('Store_Ratio')['Store_Presence'].transform('max')

In [113]:
train, test, features = split(df)

In [114]:
trn, val = train_test_split(train, test_size = 0.2, random_state = 1, stratify = train[target])

#### Input for model
X_trn, X_val = trn[features], val[features]

#### Target column
y_trn, y_val = trn[target], val[target]

#### Features for test data that we will be predicting
X_test = test[features]

In [115]:
ext = ExtraTreesClassifier(random_state = 1)
_ = ext.fit(X_trn, y_trn)

preds_val = ext.predict_proba(X_val)

log_loss(y_val, preds_val)

0.3786987727318551

In [116]:
ext = XGBClassifier(random_state = 1)
_ = ext.fit(X_trn, y_trn)

preds_val = ext.predict_proba(X_val)

log_loss(y_val, preds_val)

C:\Users\sunil\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:05:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.38677443808754336